In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
local_path = '/Users/fabienaugsburger/myCloud/Unil/Master/Thesis/Datasets/Curnagl/ERA5_1990_2m_dewpoint_temperature.nc'
servor_path = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/fabien2/ERA5/'

print(servor_path+'2m_dewpoint/')

/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/fabien2/ERA5/2m_dewpoint/


In [3]:
# open all the 2dm dewpoint temperature files

#try_0 = xr.open_dataset('/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/fabien2/ERA5/2m_dewpoint/ERA5_1990_2m_dewpoint_temperature.nc')

d2m = xr.open_mfdataset(servor_path+'2m_dewpoint/*.nc', combine='by_coords')
d2m

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 84728)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1990-01-01 ... 2019-12-31T21:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(2920, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-10-13 21:02:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [4]:
# import all tracks

track_dict = {}

for i in range(1, 97):
    # Read the CSV file and store it in the dictionary
    track_dict[f'track_{i-1}'] = pd.read_csv(f'tc_irad_tracks/tc_irad_{i}.txt')

# create a list of all the tracks

track_list = []

for i in range(1, 97):
    track_list.append(track_dict[f'track_{i-1}'])

In [5]:
# slice the time dimension by storm

storm_data = {}

dates = pd.read_csv('storms_start_end.csv', sep=',')

for i in range(0,96):
    start_first_temp = dates['start_date'][i]
    end_first_temp = dates['end_date'][i]

    d2m_storm_i = d2m.sel(time=slice(start_first_temp, end_first_temp))

    # Store the selected DataArray in the dictionary
    storm_data[f'storm_{i}'] = d2m_storm_i

storm_data

{'storm_0': <xarray.Dataset>
 Dimensions:    (longitude: 1440, latitude: 721, time: 62)
 Coordinates:
   * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
   * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
   * time       (time) datetime64[ns] 1990-01-21T03:00:00 ... 1990-01-28T18:00:00
 Data variables:
     d2m        (time, latitude, longitude) float32 dask.array<chunksize=(62, 721, 1440), meta=np.ndarray>
 Attributes:
     Conventions:  CF-1.6
     history:      2023-10-13 21:02:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...,
 'storm_1': <xarray.Dataset>
 Dimensions:    (longitude: 1440, latitude: 721, time: 52)
 Coordinates:
   * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
   * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
   * time       (time) datetime64[ns] 1990-01-23T03:00:00 ... 1990-01-29T12:00:00
 Data variables:
     d2m        (time, latitude, longitude) f

In [7]:
# iterate through each time step

import dask.array as da

var_out = []

for j in range(0, 96):
    print(f"Processing track_{j}")
    for i in range(0, len(track_dict[f'track_{j}'])):
        print(f"  Processing index {i}")
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_dict[f'track_{j}'].iloc[i]

        lon_test = np.asanyarray(d2m.longitude[:])
        lat_test = np.asanyarray(d2m.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        temp_ds = da.asarray(d2m['d2m'])

        var_out.append(temp_ds[i,closest_lat_n:closest_lat_s,closest_lon_e:closest_lon_w])

Processing track_0
  Processing index 0
  Processing index 1
  Processing index 2
  Processing index 3
  Processing index 4
  Processing index 5
  Processing index 6
  Processing index 7
  Processing index 8
  Processing index 9
  Processing index 10
  Processing index 11
  Processing index 12
  Processing index 13
  Processing index 14
  Processing index 15
  Processing index 16
  Processing index 17
  Processing index 18
  Processing index 19
  Processing index 20
  Processing index 21
  Processing index 22
  Processing index 23
  Processing index 24
  Processing index 25
  Processing index 26
  Processing index 27
  Processing index 28
  Processing index 29
  Processing index 30
  Processing index 31
  Processing index 32
  Processing index 33
  Processing index 34
  Processing index 35
  Processing index 36
  Processing index 37
  Processing index 38
  Processing index 39
  Processing index 40
  Processing index 41
  Processing index 42
  Processing index 43
  Processing index 44
 

In [33]:
# check if each square has the same shape

for i in range(0, len(var_out)):
    #print(var_out[i].shape)
    if var_out[i].shape != (32,32):
        print(f"Shape of index {i} is {var_out[i].shape}")

        # not every square is square so to check next time :(

Shape of index 81 is (32, 26)
Shape of index 82 is (32, 0)
Shape of index 83 is (32, 0)
Shape of index 156 is (32, 31)
Shape of index 157 is (32, 29)
Shape of index 158 is (32, 27)
Shape of index 159 is (32, 27)
Shape of index 160 is (32, 25)
Shape of index 161 is (32, 23)
Shape of index 162 is (32, 22)
Shape of index 163 is (32, 19)
Shape of index 164 is (32, 0)
Shape of index 165 is (32, 0)
Shape of index 166 is (32, 0)
Shape of index 167 is (32, 0)
Shape of index 168 is (32, 0)
Shape of index 188 is (32, 28)
Shape of index 209 is (32, 23)
Shape of index 210 is (32, 0)
Shape of index 278 is (32, 30)
Shape of index 279 is (32, 15)
Shape of index 280 is (32, 0)
Shape of index 325 is (32, 28)
Shape of index 326 is (32, 17)
Shape of index 327 is (32, 0)
Shape of index 372 is (32, 31)
Shape of index 373 is (32, 21)
Shape of index 374 is (32, 0)
Shape of index 375 is (32, 0)
Shape of index 427 is (29, 32)
Shape of index 428 is (28, 32)
Shape of index 429 is (28, 32)
Shape of index 430 is (

In [32]:
bug = var_out[446]

bug = bug.compute()

In [26]:
# check the coordinates of var_out that are not 32x32

for i in range(0, len(var_out)):
    if var_out[i].shape != (32,32):
        print(f"Coordinates of index {i} are {track_dict[f'track_{i}'].iloc[i]}")

IndexError: single positional indexer is out-of-bounds

In [13]:
# convert from dask array to numpy array

var_out_np = []

for i in range(0, len(var_out)):
    var_out_np.append(var_out[i].compute())
    

In [23]:
# Check if any array is empty
np.isnan(var_out_np)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4890,) + inhomogeneous part.

In [15]:
# mean, min, max value of each time step

mean_out_d2m = []
min_out_d2m = []
max_out_d2m = []

for i in range(0, len(var_out_np)):
    mean_out_d2m.append(np.mean(var_out_np[i]))
    min_out_d2m.append(np.min(var_out_np[i]))
    max_out_d2m.append(np.max(var_out_np[i]))

/users/faugsbur/miniconda3/envs/kera_lgbm/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/users/faugsbur/miniconda3/envs/kera_lgbm/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity

In [12]:
# save as csv

df_mean = pd.DataFrame(mean_out_d2m)
df_min = pd.DataFrame(min_out_d2m)
df_max = pd.DataFrame(max_out_d2m)

df_mean.to_csv('d2m_mean.csv')
df_min.to_csv('d2m_min.csv')
df_max.to_csv('d2m_max.csv')

ValueError: zero-size array to reduction operation minimum which has no identity

For all variables, (class by variable (folder), then by storm)) 